In [ ]:
import pandas as pd

prev_tp = 374
prev_tot = 3202
tot_gold = 660
K = 30

second_tp_tab = pd.read_csv("../../output/debug/true_positive_second.csv")
second_fp_tab = pd.read_csv("../../output/debug/false_positive_second.csv")
second_tab = pd.concat([second_tp_tab, second_fp_tab], ignore_index=True)

# sort
second_tab = second_tab.sort_values(by=["cosine"], ascending=False)
slim_tab = second_tab.head(K)

# calculate recall
gold_tab = slim_tab[slim_tab["label"] == 1]
tp = prev_tp + len(gold_tab)
tot = prev_tot + K

# report
recall = tp * 1.0 / tot_gold
precision = tp * 1.0 / tot
F1 = 2 * recall * precision / (recall + precision)

print(f"recall : {recall}")
print(f"precision : {precision}")
print(f"F1 : {F1}")

recall : 0.5696969696969697
precision : 0.11669770328988206
F1 : 0.19371458011334367


In [ ]:
import gensim.models
from gensim import utils
import pandas as pd
import numpy as np
import time

# read the fp results
# too small corpus
fp_tab = pd.read_csv("../../output/debug/false_positive_second.csv")
# feed entire tables
tab_A = pd.read_csv("../../output/buffer/clean_A.csv")
tab_B = pd.read_csv("../../output/buffer/clean_B.csv")

l_title = tab_A["title"].tolist()
r_title = tab_B["title"].tolist()
fp_l_title = fp_tab["ltable_title"].tolist()
fp_r_title = fp_tab["rtable_title"].tolist()

# get corpus
raw_text = l_title + r_title
    
# tokenize
corpus = []
for i, line in enumerate(raw_text):
    toks = utils.simple_preprocess(line)
    corpu = gensim.models.doc2vec.TaggedDocument(toks, [i])
    corpus.append(corpu)
    
# train
st = time.time()
doc2vec = gensim.models.doc2vec.Doc2Vec(vector_size=128, min_count=2, epochs=40)
doc2vec.build_vocab(corpus)
doc2vec.train(corpus, total_examples=doc2vec.corpus_count, epochs=doc2vec.epochs)
print(f"corpus size : {len(corpus)}, training time : {time.time() - st}")

# prepare vectors
pro_l_title = [utils.simple_preprocess(doc) for doc in fp_l_title]
pro_r_title = [utils.simple_preprocess(doc) for doc in fp_r_title]
vec_l_title = [doc2vec.infer_vector(doc) for doc in pro_l_title]
vec_r_title = [doc2vec.infer_vector(doc) for doc in pro_r_title]
    
# new cosine
new_cos_val = [np.dot(l_vec, r_vec) / (np.linalg.norm(l_vec) * np.linalg.norm(r_vec)) \
               for _, (l_vec, r_vec) in enumerate(zip(vec_l_title, vec_r_title))]

# save to disk
fp_tab["new_cosine"] = new_cos_val
fp_tab["diff_cosine"] = fp_tab["cosine"] - fp_tab["new_cosine"]
fp_tab.to_csv("../../output/debug/false_positive_second.csv", index=False)

corpus size : 4589, training time : 10.042770385742188


: 

In [2]:
# check the value distribution in tp & fp matching results
import pandas as pd

path_fp = "../../output/exp/match_res_slim.csv"
path_neg_vec = "../value_matcher/ic_values/vec_interchangeable_neg.txt"

fp = pd.read_csv(path_fp)
title_collections = fp["ltable_title"].tolist() + fp["rtable_title"].tolist()

def extract_strings(file_path):
    strings = []
    with open(file_path, 'r') as file:
        total_strings = int(file.readline().strip())
        for _ in range(total_strings):
            string = file.readline().strip()
            strings.append(string)
            num_vectors = int(file.readline().strip())
            for _ in range(num_vectors):
                file.readline()  # Skip the vector lines
    return strings

neg_title_collections = extract_strings(path_neg_vec)

title_collections = set(title_collections)
neg_title_collections = set(neg_title_collections)

print(title_collections.issubset(neg_title_collections))
print(neg_title_collections.difference(title_collections))

False
{'lock on modern air combat 7.49', 'individual software inc systemsuite 8 professional us pos', 'high impact email marketing 3.0', 'symantec backup exec 11d for windows servers agent for lotus domino server complete package 772.14', 'lotus dev . ibm db2 net search extender v. 8.2 media bb0kyna lotus development corp', '98741h mobile phonetools v. 4.0 complete package 1 user cd win', 'calendar creator 10 broderbund', 'vrs4 .0 plus scsi workgroup scanning 5 6', 'cosmic blobs lab rat edition win mac 39.99', 'adobe solution expert support 299.0', 'jumpstart advanced preschool v2 .0 knowledge adventure', 'global software a8658jch smart steps kinder win 95 98 me 2000 xp vista 9.08', 'sony creative software vegas43 dvd production suite v. 7.0 complete product', 'foreign policy reform winmac jewel case', 'symantec backup exec system recovery server edition business pack v. 7.0 complete package 1 year basic maintenance 1 user cd win multi', 'microsoft visual studio professional wmsdn prof